# End-to-end Sequence Labeling via Bi-directional LSTM-CNNs-CRF

##### This paper introduces a novel neutral network architecture, by using combination of bidirectional LSTM, CNN and CRF. This system is truly end-to-end, requiring no feature-engineering  or data pre-processig, thus making it applicable to a wide range of sequence labeling tasks

In order to run the following cells, users need to download embedding file of [Glove](https://nlp.stanford.edu/projects/glove/), which gives us the word embedding and character embedding vectors. According to the paper's instruction, we used the `glove.6B.100d.txt` embedding file. Then, the embedding file should be put under the `./data/` directory.

After setting the embedding file, then we need to set the training, validation and testing file. In this experiment (demonstration), we will use `Conll 2003` benchmark datasets, you can download the three files [here](https://www.kaggle.com/alaakhaled/conll003-englishversion). After downloading the three files, please put them under the `./data/` directory.

 After setting all the required files, we need to do some setting in the `configs.py` file, there, you need to make sure the `GLOVE_EMBEDDINGS`, `TRAINING_FILE`, `VALIDATION_FILE`, `TEST_FILE` parameters all point correctly to the four files above

According to the original repo's requirements, some packages version are : python - 3.6.6; keras(tensorflow backend) - 2.2.2 ; tensorflow - 1.10.0; keras_contrib - 2.0.8; Numpy - 1.14.2; seqeval - 0.0.5

## Import the libraries and some dependent files

In [1]:
from keras.callbacks import Callback
import prepare_data as prd
import configs
import numpy as np
from seqeval.metrics import f1_score, classification_report
from keras.callbacks import ModelCheckpoint
import model as mdl

Using TensorFlow backend.


In [2]:
from paper1_modified import End_to_end as ner

Prepare the training and validation files

In [3]:
filenames = [configs.TRAINING_FILE, configs.VALIDATION_FILE]

Make sure the MAX_SEQ_LEN is bigger than the length of the longest sentence in the training and validation files

In [4]:
print(configs.MAX_SEQ_LEN)

150


## Read the training and validation files

The created embedding files will be saved under `./data/` directory, so that when reading and processing the testing file later, we will use the same embedding method

In [5]:
data1 = ner.read_dataset(filenames)

Total no of input sequences: 7532


## Train the model

The model architecture is shown below, and you can check the training results at each epoch.

#### Here in order to go through this demo while saving time, I have set the epoch number to be 1 in the `./configs.py` file, so the F1-score is not very good, but in your experiment, you can change the epoch number to get a better trained model (or result) .

In [6]:
ner.train(data1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\konoha\Anaconda3\lib\site-packages\keras_contrib-2.0.8-py3.7.egg\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
C:\Users\konoha\Anaconda3\lib\site-packages\keras_contrib-2.0.8-py3.7.egg\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, 150, 40)      0                                            
__________________________________________________________________________________________________
char_emb (Embedding)            (None, 150, 40, 30)  2520        char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 150, 40, 30)  2730        char_emb[0][0]                   
__________________________________________________________________________________________________
word_input (InputLayer)         (None, 150)          0                                            
__________________________________________________________________________________________________
time_distr

## Predicting the NER labels

Aftre the training is done, the trained model will be saved as `model_weights.hdf5` file, and you can predict the NER labels for the testing ot deployment file with it by invoking the predict() method. The testing file will be read inside this method.

I have print out the output contents for the testing file, they have three columns, being `(current word, correct labels, predicted labels)`

In [7]:
data2 = ner.predict()

Total no of test sequences:  3454


C:\Users\konoha\Anaconda3\lib\site-packages\keras_contrib-2.0.8-py3.7.egg\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
C:\Users\konoha\Anaconda3\lib\site-packages\keras_contrib-2.0.8-py3.7.egg\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy


WORD TRUE_LABEL PRED_LABEL



SOCCER O O

- O O

JAPAN B-LOC B-PER

GET O O

LUCKY O O

WIN O O

, O O

CHINA B-PER B-ORG

IN O O

SURPRISE O O

DEFEAT O O

. O O



Nadim B-PER B-ORG

Ladki I-PER O



AL-AIN B-LOC B-ORG

, O O

United B-LOC B-LOC

Arab I-LOC B-MISC

Emirates I-LOC B-LOC

1996-12-06 O O



Japan B-LOC B-LOC

began O O

the O O

defence O O

of O O

their O O

Asian B-MISC B-MISC

Cup I-MISC I-MISC

title O O

with O O

a O O

lucky O O

2-1 O O

win O O

against O O

Syria B-LOC B-LOC

in O O

a O O

Group O O

C O O

championship O O

match O O

on O O

Friday O O

. O O



But O O

China B-LOC B-LOC

saw O O

their O O

luck O O

desert O O

them O O

in O O

the O O

second O O

match O O

of O O

the O O

group O O

, O O

crashing O O

to O O

a O O

surprise O O

2-0 O O

defeat O O

to O O

newcomers O O

Uzbekistan B-LOC O

. O O



China B-LOC B-LOC

controlled O O

most O O

of O O

the O O

match O O

and O O

saw O O

several O O

chances O O

missed O O

u


on O O

Friday O O

. O O



Takuya B-PER O

Takagi I-PER O

headed O O

the O O

winner O O

in O O

the O O

88th O O

minute O O

of O O

the O O

group O O

C O O

game O O

after O O

goalkeeper O O

Salem B-PER B-PER

Bitar I-PER I-PER

spoiled O O

a O O

mistake-free O O

display O O

by O O

allowing O O

the O O

ball O O

to O O

slip O O

under O O

his O O

body O O

. O O



It O O

was O O

the O O

second O O

Syrian B-MISC B-MISC

defensive O O

blunder O O

in O O

four O O

minutes O O

. O O



Defender O O

Hassan B-PER B-PER

Abbas I-PER I-PER

rose O O

to O O

intercept O O

a O O

long O O

ball O O

into O O

the O O

area O O

in O O

the O O

84th O O

minute O O

but O O

only O O

managed O O

to O O

divert O O

it O O

into O O

the O O

top O O

corner O O

of O O

Bitar B-PER B-ORG

's O O

goal O O

. O O



Syria B-LOC B-LOC

had O O

taken O O

the O O

lead O O

from O O

their O O

first O O

serious O O

attack O O

in O O

the O O

seventh O O



Extras O O

( O O

b-9 O O

lb-3 O O

w-12 O O

nb-2 O O

) O O

26 O O



Total O O

( O O

all O O

out O O

in O O

42.1 O O

overs O O

) O O

231 O O



Fall O O

of O O

wickets O O

: O O

1-3 O O

2-7 O O

3-125 O O

4-146 O O

5-170 O O

6-190 O O

7-195 O O



8-213 O O

9-216 O O

. O O



Bowling O O

: O O

Wasim B-PER B-PER

Akram I-PER I-PER

8.1-0-43-3 O O

( O O

9w O O

, O O

1nb O O

) O O

, O O

Waqar B-PER B-PER

Younis I-PER I-PER



6-0-32-2 O O

( O O

2w O O

, O O

1nb O O

) O O

, O O

Saqlain B-PER B-PER

Mushtaq I-PER I-PER

8-0-54-2 O O

, O O

Mushtaq B-PER B-PER

Ahmad I-PER I-PER



10-0-42-0 O O

( O O

1w O O

) O O

, O O

Shahid B-PER B-PER

Afridi I-PER I-PER

7-0-40-2 O O

, O O

Salim B-PER B-PER

Malik I-PER I-PER

2.5-0-8-1 O O

, O O



Ijaz B-PER B-PER

Ahmad I-PER I-PER

0.1-0-0-0 O O

. O O



Result O O

: O O

Pakistan B-LOC B-LOC

won O O

by O O

46 O O

runs O O

. O O



Third O O

one-day O O

match O O

: O O

December O O

8 O 

...


Postponed O O

: O O

Airdrieonians B-ORG B-ORG

v O O

Clydebank B-ORG B-ORG

( O O

to O O

Wednesday O O

) O O

, O O

East B-ORG B-LOC



Fife B-ORG B-ORG

v O O

Partick B-ORG B-ORG

, O O

Stirling B-ORG B-ORG

v O O

St B-ORG B-ORG

Mirren I-ORG I-ORG

( O O

to O O

Tuesday O O

) O O



Division O O

two O O



Livingston B-ORG B-ORG

2 O O

Stenhousemuir B-ORG B-ORG

1 O O



Stranraer B-ORG B-ORG

0 O O

Brechin B-ORG B-ORG

1 O O



Division O O

three O O



Ross B-ORG B-PER

County I-ORG I-ORG

4 O O

Montrose B-ORG B-ORG

4 O O



Postponed O O

: O O

Forfar B-ORG B-ORG

v O O

Alloa B-ORG B-ORG

, O O

Inverness B-ORG B-ORG

Thistle I-ORG I-ORG

v O O

Queen B-ORG O

's I-ORG O

Park I-ORG O



Scottish B-MISC B-MISC

Cup I-MISC O

first O O

round O O



Alloa B-ORG B-ORG

3 O O

Hawick B-ORG B-ORG

1 O O



Elgin B-ORG B-ORG

City I-ORG I-ORG

0 O O

Whitehill B-ORG B-ORG

3 O O



Postponed O O

: O O

Albion B-ORG B-ORG

v O O

Forfar B-ORG B-ORG

, O O

Huntly B


Dede I-PER O

, O O

Arjan B-PER B-PER

Bellai I-PER I-ORG



Midfielders O O

- O O

Bledar B-PER B-PER

Kola I-PER I-PER

, O O

Altin B-PER B-PER

Haxhi I-PER I-PER

, O O

Sokol B-PER B-PER

Prenga I-PER I-PER

, O O

Ervin B-PER O

Fakaj I-PER O



Forwards O O

- O O

Altin B-PER B-PER

Rraklli I-PER I-PER

, O O

Viktor B-PER B-PER

Paco I-PER I-PER

, O O

Fatmir B-PER B-PER

Vata I-PER I-PER

, O O

Erjon B-PER B-PER

Bogdani I-PER I-ORG

. O O



CRICKET O O

- O O

JONES B-PER B-PER

HITS O O

CENTURY O O

AS O O

VICTORIA B-ORG I-ORG

FIGHT O O

BACK O O

. O O



HOBART B-LOC B-ORG

, O O

Australia B-LOC B-LOC

1996-12-07 O O



Former O B-MISC

Australia B-LOC B-LOC

test O O

batsman O O

Dean B-PER B-PER

Jones I-PER I-PER

hit O O

an O O

unbeaten O O

130 O O

to O O

lead O O

Victoria B-LOC B-LOC

's O O

fightback O O

in O O

their O O

Sheffield B-MISC B-ORG

Shield I-MISC I-ORG

match O O

against O O

Tasmania B-ORG O

on O O

Saturday O O

. O O



Replying


9 O O

13 O O

9 O O



Hapoel B-ORG B-ORG

Tel I-ORG I-ORG

Aviv I-ORG I-ORG

12 O O

2 O O

3 O O

7 O O

7 O O

16 O O

9 O O



SOCCER O O

- O O

ASIAN B-MISC B-ORG

CUP I-MISC O

RESULTS O O

. O O



ABU B-LOC B-ORG

DHABI I-LOC I-ORG

1996-12-07 O O



Results O O

of O O

Asian B-MISC B-MISC

Cup I-MISC I-MISC

group O O

A O O

matches O O

on O O

Saturday O O

: O O



United B-LOC B-LOC

Arab I-LOC I-ORG

Emirates I-LOC B-LOC

3 O O

Kuwait B-LOC B-LOC

2 O O

( O O

halftime O O

0-2 O O

) O O



Scorers O O

: O O



UAE B-LOC B-LOC

- O O

Hassan B-PER B-PER

Ahmed I-PER I-PER

53 O O

, O O

Adnan B-PER B-PER

Al I-PER I-PER

Talyani I-PER I-ORG

55 O O

, O O

Bakhit B-PER B-ORG

Saad I-PER I-ORG

80 O O



Kuwait B-LOC B-LOC

- O O

Jassem B-PER B-ORG

Al-Huwaidi I-PER I-ORG

9 O O

, O O

44 O O



Attendance O O

: O O

15,000 O O



South B-LOC B-LOC

Korea I-LOC B-LOC

4 O O

Indonesia B-LOC B-LOC

2 O O

( O O

3-0 O O

) O O



Scorers O O

: O O



South B-L


1 O O



Philadelphia B-LOC B-ORG

6 O O

DALLAS B-LOC B-ORG

3 O O



St B-LOC B-ORG

Louis I-LOC I-ORG

4 O O

COLORADO B-LOC B-ORG

3 O O



EDMONTON B-LOC B-ORG

5 O O

Ottawa B-LOC B-ORG

2 O O



NHL B-ORG B-ORG

ICE O O

HOCKEY O I-ORG

- O O

CANUCKS B-ORG B-PER

RW O I-ORG

BURE B-PER I-ORG

SUSPENDED O O

FOR O O

ONE O O

GAME O O

. O O



NEW B-LOC B-LOC

YORK I-LOC I-ORG

1996-12-06 O O



Vancouver B-ORG B-ORG

Canucks I-ORG I-ORG

star O O

right O O

wing O O

Pavel B-PER B-PER

Bure I-PER I-ORG

was O O

suspended O O

for O O

one O O

game O O

by O O

the O O

National B-ORG B-MISC

Hockey I-ORG I-ORG

League I-ORG O

and O O

fined O O

$ O O

1,000 O O

Friday O O

for O O

his O O

hit O O

on O O

Buffalo B-ORG B-ORG

Sabres I-ORG B-ORG

defenceman O O

Garry B-PER I-ORG

Galley I-PER I-ORG

on O O

Wednesday O O

. O O



Bure B-PER B-ORG

received O O

a O O

double-minor O O

penalty O O

for O O

high-sticking O O

with O O

2:22 O O

left O O

in O O

the


my O O

life O O

. O O



It O O

all O O

culminated O O

in O O

the O O

fact O O

that O O

I O O

now O O

have O O

lots O O

of O O

great O O

, O O

great O O

friends O O

in O O

Ireland B-LOC B-LOC

. O O



That O O

is O O

why O O

this O O

is O O

so O O

emotional O O

a O O

night O O

for O O

me O O

, O O

" O O

Charlton B-PER B-ORG

said O O

. O O



" O O

It O O

was O O

the O O

joy O O

that O O

we O O

all O O

had O O

over O O

the O O

period O O

, O O

that O O

I O O

shared O O

with O O

people O O

that O O

I O O

grew O O

to O O

love O O

, O O

that O O

I O O

treasure O O

most O O

, O O

" O O

he O O

added O O

. O O



Charlton B-PER B-ORG

managed O O

Ireland B-LOC B-LOC

for O O

93 O O

matches O O

, O O

during O O

which O O

time O O

they O O

lost O O

only O O

17 O O

times O O

in O O

almost O O

10 O O

years O O

until O O

he O O

resigned O O

in O O

December O O

1995 O O

. O O



He O O

guided O O

Ireland B-

## Evaluate the result

The evaluation metrics are `precision`, `recall`, and `F1-acore`

In [8]:
precision, recall, f1Score = ner.evaluation(data2)

Report:
              precision    recall  f1-score   support

       MISC       0.73      0.39      0.51       702
        ORG       0.43      0.67      0.53      1661
        LOC       0.70      0.63      0.66      1668
        PER       0.82      0.62      0.71      1617

avg / total       0.66      0.61      0.62      5648



#### convert_ground_truth()

The convert_ground_truth() method returns the ground labels of the testing file, they are saved in a list, the space lines is used to separate two sentences.  

In [9]:
data = ner.convert_ground_truth(configs.TEST_FILE)

In [10]:
data

['O',
 ' ',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'O',
 'O',
 'O',
 'O',
 ' ',
 'B-PER',
 'I-PER',
 ' ',
 'B-LOC',
 'O',
 'B-LOC',
 'I-LOC',
 'I-LOC',
 'O',
 ' ',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-MISC',
 'I-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 ' ',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 ' ',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-MISC',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 ' ',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 ' ',
 'O',
 'O',
 'B-MISC',
 'O',
 'O',
 'O',
 'O',
 